In [1]:
from pathlib import Path

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [2]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
DATA_DIR = Path("../resources")

In [3]:
client = BenchmarkClient(url=URL)

# Workouts

In [23]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df = df.rename(columns={"name": "routine_name"})
df.head()

,workoutroutine,date,exercise,weight,reps
0,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16
1,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16
2,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,10
3,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,8
4,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,6


In [24]:
latest_workout_date = client.workouts.get_latest_workout()["date"]
df = df[df["date"] > latest_workout_date]
df.head()

,workoutroutine,date,exercise,weight,reps


In [25]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [27]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

,exercise_name,exercise_id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [28]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines.head()

,routine_name,routine_id,routine_exercises
0,wave loading 321 lower 1,1,"[{'exercise': {'name': 'trap bar deadlift', 'i..."
1,wave loading 321 upper 1,2,"[{'exercise': {'name': 'bench press', 'id': 1}..."
2,wave loading 321 lower 2,3,"[{'exercise': {'name': 'barbell back squats', ..."
3,wave loading 321 upper 2,4,"[{'exercise': {'name': 'military press', 'id':..."
4,wave loading 642 lower 1,5,"[{'exercise': {'name': 'trap bar deadlift', 'i..."


In [29]:
df["exercise_lower"] = df.exercise.str.lower()
df["workoutroutine_lower"] = df.workoutroutine.str.lower()
df.head()

,workoutroutine,date,exercise,weight,reps,exercise_lower,workoutroutine_lower


In [30]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="exercise_name")
df_all = df_all.merge(right=routines, left_on="workoutroutine_lower", right_on="routine_name")[["date", "weight", "reps", "routine_id", "exercise_id"]]
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

,,,weight,reps
date,routine_id,exercise_id,,


In [ ]:
for date in dates:
    all_workouts = []
    routines = df_all.loc[date].index.get_level_values(0).unique()
    for routine in routines:
        all_exercises = []
        exercises = df_all.loc[(date, routine)].index.unique()
        for exercise in exercises:
            sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
            data = {"id": exercise, "sets": sets}
            all_exercises.append(data)
        try:
            response = client.workouts.create_workout(date=str(date), exercises=all_exercises, workoutroutine_id=routine)
            print(f"Created Workout for date {date}")
        except Exception as e:
            print(e)